# Fase 1: Importar las dependencias


In [1]:
import pandas as pd
import nltk

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fuente del dataset: https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

In [3]:
t_news = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Predicion fake news/True.csv')
f_news = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Predicion fake news/Fake.csv')

print(t_news.shape)
print(f_news.shape)

total_nan_values_t = t_news.isnull().sum().sum()
print (total_nan_values_t)
total_nan_values_f = f_news.isnull().sum().sum()
print (total_nan_values_f)

(21417, 4)
(23481, 4)
0
0


In [4]:
t_news.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
f_news.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
#Uniremos la columna Titulo con la columna del Texto para no perder esa información

t_news["news"] = t_news["title"] + " " + t_news["text"]
t_news.drop(['title', 'text', 'subject', 'date'], axis=1, inplace=True)
t_etiqueta = t_news.assign(Etiqueta=1)
t_etiqueta.head(5)

,news,Etiqueta
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [7]:
f_news["news"] = f_news["title"] + " " + f_news["text"]
f_news.drop(['title', 'text', 'subject', 'date'], axis=1, inplace=True)
f_etiqueta = f_news.assign(Etiqueta=0)
f_etiqueta.head(5)

,news,Etiqueta
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0


In [8]:
news = pd.concat([t_etiqueta, f_etiqueta], axis=0)
news.reset_index(drop=True, inplace=True)
news

,news,Etiqueta
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1
...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0
44896,How to Blow $700 Million: Al Jazeera America F...,0


In [9]:
from sklearn.utils import shuffle

In [11]:
news = shuffle(news)
news

,news,Etiqueta
34046,BUSTED! ABC/WASHINGTON POST POLL Showing Hilla...,0
26718,WATCH: Donald Trump Tells Russia To Commit Es...,0
30382,Fox News Host Megyn Kelly Completely Embarras...,0
28916,North Carolina Authorities May Charge Trump W...,0
1919,Trump says U.S. considering global trade embar...,1
...,...,...
2510,U.S. senators seek ban on pesticide chlorpyrif...,1
7754,"'Absolutely false,' Trump says defiantly after...",1
36164,IS OBAMA’S RADICAL AGENDA Responsible For Stat...,0
10629,Trump and Christie laid foundation of their al...,1


## Pre Procesado

### Limpieza

In [12]:
import string

In [13]:
#para sacar las palabras no importantes

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
#vamos a crear una función de limpieza

def procesar_texto(cadena):
    sinpunt = [c for c in cadena if c not in string.punctuation]
    sinpunt = ''.join(sinpunt)
    palabras = sinpunt.split()
    cadena_cleaned = [palabra for palabra in palabras if palabra.lower() not in stopwords.words('english')]
    return cadena_cleaned

In [15]:
#aplicaremos la función sobre df

news['news'].head(5).apply(procesar_texto)

34046    [BUSTED, ABCWASHINGTON, POST, POLL, Showing, H...
26718    [WATCH, Donald, Trump, Tells, Russia, Commit, ...
30382    [Fox, News, Host, Megyn, Kelly, Completely, Em...
28916    [North, Carolina, Authorities, May, Charge, Tr...
1919     [Trump, says, US, considering, global, trade, ...
Name: news, dtype: object

In [16]:
#SEGUNDO PASO = vectorizar

from sklearn.feature_extraction.text import CountVectorizer

In [17]:
transformador = CountVectorizer(analyzer=procesar_texto).fit(news['news'])

In [18]:
# - tengo que imporar módulos de ML

from sklearn.feature_extraction.text import TfidfTransformer
# tranformador de los mensajes

from sklearn.ensemble import RandomForestClassifier
# algoritmo de clasificacion

from sklearn.model_selection import train_test_split
# dividir los datos de entrenamiento y test

from sklearn.pipeline import Pipeline
# para ejecutar operaciones simultaneamente

from sklearn.metrics import classification_report
# para hacer el informa de clasificacion de este algoritmo


In [25]:
# PRIMERO : dividir mensajes de entrenamiento y mensajes de prueba, con etiquetas

# genero cuatro variables correspondietnes 

com_entr, com_pru, etq_entr, etq_pru = train_test_split(news['news'], news['Etiqueta'], test_size=0.3)

In [26]:
print(com_entr.shape)
print(com_pru.shape)
print(etq_entr.shape)
print(etq_pru.shape)

(31428,)
(13470,)
(31428,)
(13470,)


In [27]:
pipeline = Pipeline([
    ('vectorizar', CountVectorizer(analyzer=procesar_texto)), #primera operación
    ('transformar', TfidfTransformer()),
    ('clasificar', RandomForestClassifier())
])

In [28]:
# SEGUNDO : entrenar el pipeline con los mensajes de entrenamiento

pipeline.fit(com_entr, etq_entr)

Pipeline(steps=[('vectorizar',
                 CountVectorizer(analyzer=<function procesar_texto at 0x7fe7520e3560>)),
                ('transformar', TfidfTransformer()),
                ('clasificar', RandomForestClassifier())])

In [29]:
# TERCERO : una vez entrenado el sistema, lo pruebo en sus predicciones

predicciones = pipeline.predict(com_pru)

In [30]:
# CUARTO : informe de prueba

print(classification_report(etq_pru, predicciones))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      6990
           1       0.99      1.00      0.99      6480

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470

